In [1]:
import nfl_func
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2024-08-09 17:05:34.201504: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 17:05:34.351946: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 17:05:34.509048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 17:05:34.630099: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 17:05:34.666631: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-09 17:05:34.933697: I tensorflow/core/platform/cpu_feature_gu

# Data Preparation 

When creating an AI model curating the data to train the model is most important consideration. Using meaningful variables while taking in to account different facets of the game can create a powerful model. 

## Player-Specific Variables
### General Player Information
    - Player ID: Unique identifier for each player
    - Position: Position played (QB, WR, RB, TE)
    - Team: Player's team
    - Height, Weight, Age: Physical attributes

In [2]:
roster_columns = ['player_id', 'player_name', 'first_name', 'last_name', 'team', 'position', 'depth_chart_position', 'jersey_number', 'status', 'birth_date', 'height', 'weight', 'years_exp']
roster_data = nfl_func.get_roster_data(2023, roster_columns)
roster_data

,player_id,player_name,first_name,last_name,team,position,depth_chart_position,jersey_number,status,birth_date,height,weight,years_exp
0,00-0017724,Bernard Williams,Bernard,Williams,PHI,OL,T,74.0,CUT,None,80.0,286,29
1,00-0022531,Jason Peters,Jason,Peters,SEA,OL,T,70.0,INA,1982-01-22,76.0,328,19
2,00-0023459,Aaron Rodgers,Aaron,Rodgers,NYJ,QB,QB,8.0,INA,1983-12-02,74.0,225,18
3,00-0023853,Matt Prater,Matt,Prater,ARI,K,K,5.0,ACT,1984-08-10,70.0,201,17
4,00-0024243,Marcedes Lewis,Marcedes,Lewis,CHI,TE,TE,84.0,ACT,1984-05-19,78.0,267,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3085,00-0039188,Lachlan Pitts,Lachlan,Pitts,CHI,TE,TE,81.0,CUT,None,78.0,255,0
3086,00-0039189,Micah Vanterpool,Micah,Vanterpool,NE,OL,T,72.0,CUT,None,78.0,315,0
3087,00-0039191,Aaron Dykes,Aaron,Dykes,MIN,RB,RB,46.0,CUT,None,71.0,202,0
3088,00-0039192,Caleb Sanders,Caleb,Sanders,PHI,DL,DT,67.0,CUT,None,73.0,270,0


In [3]:
all_players = nfl_func.get_players()
player_columns = ['gsis_id', 'display_name', 'first_name', 'last_name', 'team_abbr', 'current_team_id', 'position_group', 'position', 'status', 'birth_date', 'height', 'weight', 'years_of_experience', 'college_name', 'entry_year', 'rookie_year', 'draft_number', 'college_conference']
all_players = all_players[player_columns]
all_players

,gsis_id,display_name,first_name,last_name,team_abbr,current_team_id,position_group,position,status,birth_date,height,weight,years_of_experience,college_name,entry_year,rookie_year,draft_number,college_conference
0,00-0004866,'Omar Ellison,'Omar,Ellison,LAC,4400,WR,WR,RET,1971-10-08,73.0,200.0,2,None,NaN,NaN,NaN,None
1,00-0032889,A'Shawn Robinson,A'Shawn,Robinson,CAR,0750,DL,DT,ACT,1995-03-21,76.0,330.0,9,Alabama,2016.0,2016.0,46.0,Southeastern Conference
2,00-0037845,A.J. Arcuri,A.J.,Arcuri,LA,2510,OL,T,ACT,1997-08-13,79.0,320.0,2,Michigan State,2022.0,2022.0,261.0,Big Ten Conference
3,00-0039793,A.J. Barner,A.J.,Barner,SEA,4600,TE,TE,ACT,2002-05-03,78.0,251.0,0,Michigan,2024.0,2024.0,121.0,Big Ten Conference
4,00-0030228,A.J. Bouye,Arlandus,Bouye,CAR,0750,DB,CB,RES,1991-08-16,72.0,191.0,8,Central Florida,2013.0,2013.0,NaN,American Athletic Conference
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20731,00-0039689,Zuri Henry,Zuri,Henry,NE,3200,OL,T,ACT,2000-04-12,78.0,293.0,0,Texas-El Paso,2024.0,2024.0,NaN,Conference USA
20732,00-0022024,Zuriel Smith,Zuriel,Smith,NE,3200,WR,WR,RET,1980-01-15,71.0,174.0,2,None,NaN,NaN,NaN,None
20733,00-0030855,Zurlon Tipton,Zurlon,Tipton,IND,2200,RB,RB,CUT,1989-04-27,72.0,230.0,2,Central Michigan,2014.0,2014.0,NaN,Mid-American Conference
20734,00-0037373,Zyon Gilbert,Zyon,Gilbert,GB,1800,DB,CB,CUT,1999-02-04,72.0,174.0,1,Florida Atlantic,2022.0,2022.0,NaN,Conference USA


### Historical Performance Metrics
    - Games Played: Number of games played
    - Games Started: Number of games started
    - Snaps Played: Number of snaps played
    - Passing Yards (QB): Total passing yards
    - Passing Touchdowns (QB): Total passing touchdowns
    - Interceptions (QB): Number of interceptions thrown
    - Rushing Yards (RB/QB/WR/TE): Total rushing yards
    - Rushing Touchdowns (RB/QB/WR/TE): Total rushing touchdowns
    - Receptions (WR/TE/RB): Number of receptions
    - Receiving Yards (WR/TE/RB): Total receiving yards
    - Receiving Touchdowns (WR/TE/RB): Total receiving touchdowns
    - Targets (WR/TE/RB): Number of targets
    - Fumbles: Number of fumbles

In [4]:
years = [2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
weekly_data = nfl_func.get_weekly_data(years)
weekly_columns = ['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'recent_team', 'season', 'week', 'season_type', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_2pt_conversions', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards','receiving_yards_after_catch', 'receiving_first_downs', 'receiving_2pt_conversions', 'target_share', 'air_yards_share', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr']
weekly_data[weekly_columns]

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,recent_team,season,week,season_type,opponent_team,...,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_2pt_conversions,target_share,air_yards_share,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000007,None,Rabih Abdullah,RB,RB,NE,2004,2,REG,ARI,...,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.4,0.4
1,00-0000007,None,Rabih Abdullah,RB,RB,NE,2004,5,REG,MIA,...,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,6.4,6.4
2,00-0000007,None,Rabih Abdullah,RB,RB,NE,2004,6,REG,SEA,...,0.0,0.0,0.0,0.0,0,0.034483,NaN,0.0,0.9,1.9
3,00-0000007,None,Rabih Abdullah,RB,RB,NE,2004,10,REG,BUF,...,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,-0.5,-0.5
4,00-0000007,None,Rabih Abdullah,RB,RB,NE,2004,16,REG,NYJ,...,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5626,00-0038134,K.Walker,Kenneth Walker,RB,RB,SEA,2022,17,REG,NYJ,...,0.0,-1.0,10.0,1.0,0,0.035714,-0.003717,0.0,14.2,15.2
5627,00-0038134,K.Walker,Kenneth Walker,RB,RB,SEA,2022,18,REG,LA,...,0.0,-1.0,11.0,0.0,0,0.035714,-0.003745,0.0,12.4,13.4
5628,00-0038134,K.Walker,Kenneth Walker,RB,RB,SEA,2022,19,POST,SF,...,0.0,0.0,3.0,0.0,0,0.029412,0.000000,0.0,12.6,13.6
5629,00-0038147,D.Baldwin,Daylen Baldwin,WR,WR,CLE,2022,15,REG,BAL,...,0.0,25.0,0.0,2.0,0,0.076923,0.223214,0.0,2.5,4.5


In [5]:
pbp_data = nfl_func.get_pbp_data(years)

2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [6]:
pbp_data

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
0,1.0,2004_01_ARI_STL,2004091208,LA,ARI,REG,1,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35.0,2004_01_ARI_STL,2004091208,LA,ARI,REG,1,ARI,away,LA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68.0,2004_01_ARI_STL,2004091208,LA,ARI,REG,1,ARI,away,LA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,93.0,2004_01_ARI_STL,2004091208,LA,ARI,REG,1,ARI,away,LA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,112.0,2004_01_ARI_STL,2004091208,LA,ARI,REG,1,ARI,away,LA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900912,4035.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,None,None,None,...,,,0.0,0.0,NaN,NaN,None,None,None,None
900913,4024.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,KC,away,PHI,...,00-0030615;00-0034752;00-0036230;00-0036623;00...,00-0036813;00-0037073;00-0036917;00-0037615;00...,11.0,11.0,NaN,NaN,None,None,None,None
900914,4050.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,PHI,home,KC,...,00-0035157;00-0037198;00-0037819;00-0038043;00...,00-0036813;00-0036919;00-0033251;00-0036917;00...,11.0,11.0,NaN,NaN,None,None,None,None
900915,4072.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,PHI,home,KC,...,00-0036271;00-0034351;00-0032954;00-0036919;00...,00-0036374;00-0035157;00-0035625;00-0037819;00...,11.0,11.0,NaN,6.073,False,None,ZONE_COVERAGE,PREVENT


In [7]:
weekly_data = nfl_func.get_weekly_data(2023)

features = weekly_data[['completions', 'attempts','passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards','passing_yards_after_catch', 'passing_first_downs','carries', 'rushing_yards','rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost','rushing_first_downs','receptions', 'targets', 'receiving_yards', 'receiving_tds','receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards','receiving_yards_after_catch', 'receiving_first_downs','target_share', 'air_yards_share', 'special_teams_tds']]
labels = weekly_data[['fantasy_points', 'fantasy_points_ppr']]

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display the model summary
model.summary()

print(X_train)
print('----------------------------------------------------------------------')
print(y_train)

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)



Downcasting floats.


/home/ncote/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1723244808.904213 2179566 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-09 17:06:48.905341: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,097 (16.00 KB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 0 (0.00 B)

[[-0.328424   -0.33279446 -0.32309136 ...  1.00592738  1.13521201
  -0.05769017]
 [-0.328424   -0.33279446 -0.32309136 ...  0.62974858  1.1805179
  -0.05769017]
 [-0.328424   -0.33279446 -0.32309136 ... -1.10429829 -0.73081248
  -0.05769017]
 ...
 [-0.328424   -0.33279446 -0.32309136 ...  1.28111226  0.86768942
  -0.05769017]
 [-0.328424   -0.33279446 -0.32309136 ... -0.09677263 -1.365923
  -0.05769017]
 [-0.328424   -0.33279446 -0.32309136 ...  0.35013347  0.13881959
  -0.05769017]]
----------------------------------------------------------------------
      fantasy_points  fantasy_points_ppr
3483           11.90           17.900000
4862            2.90            4.900000
1618            0.30            1.300000
2577           15.80           17.799999
897             2.30            3.300000
...              ...                 ...
3772           12.24           12.240000
5191            6.70           10.700000
5226            4.60           10.600000
5390            6.20          

In [8]:
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Display predictions
print(predictions[:10])

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan
Test Loss: nan
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]


### Team-Specific Variables
    - Team Offensive Metrics: Overall offensive performance of the team (total yards, points scored, etc.)
    - Team Defensive Metrics: Overall defensive performance of the team (yards allowed, points allowed, etc.)
    - Offensive Line Strength: Metrics like sacks allowed, pressures allowed, etc.
    - Defensive Line Strength: Metrics like sacks made, pressures applied, etc.
    - Injuries: Current injury report for the team

### Opponent-Specific Variables
    - Opponent Team Defensive Metrics: Performance metrics of the opposing team's defense (yards allowed, points allowed, turnovers forced, etc.)
    - Opponent Defensive Schemes: Information about the defensive schemes commonly used by the opponent (e.g., 4-3 defense, zone coverage tendencies, etc.)
    - Matchup History: Historical performance of the player/team against the upcoming opponent
    - Opponent Injuries: Current injury report for the opposing team

### Coaching and Strategy Variables
    - Head Coach and Coordinators: Information about the head coach, offensive coordinator, and defensive coordinator
    - Play Calling Tendencies: Historical data on play calling tendencies (run vs. pass ratio, play action usage, etc.)
    - Scheme Preferences: Offensive and defensive scheme preferences (e.g., West Coast offense, Cover 2 defense)
    - Game Plan Adjustments: Any reported adjustments or strategies for the upcoming game

### Contextual and Situational Variables
    - Game Location: Home or away game
    - Weather Conditions: Weather forecast for the game day (temperature, precipitation, wind speed, etc.)
    - Game Type: Regular season, playoff, or preseason game
    - Time of Season: Early, mid, or late season
    - Time of Game: Day game, night game, or prime-time game
    - Field Surface: Type of field surface (grass, turf, etc.)
    - Vegas Odds: Betting lines, point spreads, and over/under totals

In [9]:
import nfl_data_py as nfl
year = 2023
weekly_data = nfl.import_weekly_data([year])

roster_data = nfl.import_rosters([year])

print(type(weekly_data))

Downcasting floats.
<class 'pandas.core.frame.DataFrame'>


In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Ensure GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Example dataset
player_stats_data = {
    'Player ID': [1, 2],
    'Name': ['Player A', 'Player B'],
    'Team': ['XYZ', 'XYZ'],
    'Position': ['QB', 'WR'],
    'Games Played': [10, 10],
    'Passing Yards': [2500, 0],
    'Passing TDs': [20, 0],
    'Interceptions': [5, 0],
    'Rushing Yards': [200, 100],
    'Receptions': [0, 50],
    'Receiving Yards': [0, 600],
    'Receiving TDs': [0, 6],
    'Targets': [0, 75],
    'Fumbles': [2, 1]
}

team_stats_data = {
    'Team': ['XYZ'],
    'Offensive Yards': [4000],
    'Points Scored': [300],
    'Defensive Yards Allowed': [3500],
    'Points Allowed': [250],
    'Sacks Allowed': [15],
    'Sacks Made': [20],
    'Current Injuries': ['Player A (Out), Player B (Q)']
}

opponent_stats_data = {
    'Opponent': ['ABC'],
    'Defensive Yards Allowed': [3200],
    'Points Allowed': [200],
    'Turnovers Forced': [15],
    'Sacks Made': [18],
    'Defensive Scheme': ['4-3, Zone Coverage'],
    'Current Injuries': ['Player C (Out), Player D (Q)']
}

contextual_data = {
    'Game ID': [101],
    'Home Team': ['XYZ'],
    'Away Team': ['ABC'],
    'Location': ['Home'],
    'Weather': ['Clear'],
    'Game Type': ['Regular'],
    'Time of Season': ['Mid'],
    'Time of Game': ['Night'],
    'Field Surface': ['Turf'],
    'Vegas Odds': ['XYZ -3.5']
}

# Convert dictionaries to DataFrames
player_stats_df = pd.DataFrame(player_stats_data)
team_stats_df = pd.DataFrame(team_stats_data)
opponent_stats_df = pd.DataFrame(opponent_stats_data)
contextual_data_df = pd.DataFrame(contextual_data)

# Merge data into a single DataFrame
merged_data = pd.concat([player_stats_df, team_stats_df, opponent_stats_df, contextual_data_df], axis=1)

# Select features and target
features = merged_data[['Games Played', 'Passing Yards', 'Passing TDs', 'Interceptions', 'Rushing Yards', 
                        'Receptions', 'Receiving Yards', 'Receiving TDs', 'Targets', 'Fumbles',
                        'Offensive Yards', 'Points Scored', 'Defensive Yards Allowed', 'Points Allowed',
                        'Sacks Allowed', 'Sacks Made', 'Defensive Yards Allowed', 'Points Allowed', 
                        'Turnovers Forced', 'Sacks Made']]

target = merged_data[['Passing Yards', 'Passing TDs', 'Rushing Yards', 'Receptions', 'Receiving Yards', 'Receiving TDs']]

# Normalize the data
features = (features - features.mean()) / features.std()

# Convert to NumPy arrays
features = np.array(features)
target = np.array(target)

# Split data into training and testing sets
train_features = features[:int(0.8 * len(features))]
test_features = features[int(0.8 * len(features)):]

train_target = target[:int(0.8 * len(target))]
test_target = target[int(0.8 * len(target)):]

# Define the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[train_features.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(train_target.shape[1])
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

# Train the model
history = model.fit(train_features, train_target, epochs=100, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae, mse = model.evaluate(test_features, test_target, verbose=2)
print("Testing set Mean Abs Error: {:5.2f}".format(mae))

# Predict future games
predictions = model.predict(test_features)
print("Predictions: ", predictions)

# Continuously train with new data (dummy new data here for demonstration)
new_features = test_features
new_target = test_target

# Re-train the model with new data
model.fit(new_features, new_target, epochs=50, validation_split=0.2, verbose=1)

Num GPUs Available:  0


/home/ncote/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Training data contains 1 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.